## **Vegetable Modelling**

In [1]:
pwd

'/home/wsuser/work'

In [2]:
!pip install tensorflow

In [3]:
!pip install keras

**IMPORT NECESSARY PACKAGES**

In [4]:
from tensorflow import keras

In [5]:
import os
import cv2
import numpy as np
import tensorflow as tf

In [6]:
categories = ['Pepper,_bell___Bacterial_spot','Pepper,_bell___healthy','Potato___Early_blight','Potato___healthy','Potato___Late_blight','Tomato___Bacterial_spot','Tomato___Late_blight','Tomato___Leaf_Mold','Tomato___Septoria_leaf_spot']

**UPLOAD DATASET**

In [9]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='vE3Hbzepn_CelL3B3ukJykuZ_Iq-aAeTUtOk0kykig3e',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'vegetableclassification-donotdelete-pr-zexqx5zjyyluv9'
object_key = 'Veg-dataset.zip'

streaming_body_3 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [10]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_3.read()), 'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [11]:
data_dir = '/home/wsuser/work/Veg-dataset/train_set'

In [12]:
data = []

In [13]:
img_size = 100
def make_data():
    for category in categories:
        path = os.path.join(data_dir, category)
        label = categories.index(category)
        for img_name in os.listdir(path):
            image_path = os.path.join(path, img_name)
            image = cv2.imread(image_path)
            try:
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image,(img_size,img_size))
                image = np.array(image, dtype=np.float32)
                data.append([image,label])
            except Exception as e:
                pass  
            
make_data() 

In [14]:
len(data)

11386

In [15]:
np.random.shuffle(data)

In [16]:
features = []
labels = []
for img,label in data:
    features.append(img)
    labels.append(label)

In [17]:
features = np.array(features, dtype = np.float32)
labels = np.array(labels)
features = features/255.0

**ADD LAYERS**

In [18]:
input_layer = tf.keras.layers.Input([100,100,3])

In [19]:
conv1 = tf.keras.layers.Conv2D(filters = 64, kernel_size = (5,5), padding = 'Same', activation = 'relu')(input_layer)

In [20]:
pool1 = tf.keras.layers.MaxPooling2D(pool_size = (2,2))(conv1)

In [21]:
conv2 = tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same', activation = 'relu')(pool1)

In [22]:
pool2 = tf.keras.layers.MaxPooling2D(pool_size = (2,2), strides = (2,2))(conv2)

In [23]:
conv3 = tf.keras.layers.Conv2D(filters = 96, kernel_size = (3,3), padding = 'Same', activation = 'relu')(pool2)

In [24]:
pool3 = tf.keras.layers.MaxPooling2D(pool_size = (2,2), strides = (2,2))(conv3)

In [25]:
conv4 = tf.keras.layers.Conv2D(filters = 96, kernel_size = (3,3), padding = 'Same', activation = 'relu')(pool3)

In [26]:
pool4 = tf.keras.layers.MaxPooling2D(pool_size = (2,2), strides = (2,2))(conv4)

In [27]:
flt1 = tf.keras.layers.Flatten()(pool4)

In [28]:
dn1 = tf.keras.layers.Dense(256,activation = 'relu')(flt1)

In [29]:
dn2 = tf.keras.layers.Dense(124,activation = 'relu')(dn1)

In [30]:
out = tf.keras.layers.Dense(10,activation = 'softmax')(dn1)

**MODELING**

In [31]:
model = tf.keras.Model(input_layer, out)

In [32]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [33]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 100, 100, 64)      4864      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 50, 50, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 50, 50, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 25, 25, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 25, 25, 96)        55392 

**FIT THE MODEL**

In [34]:
model.fit(features, labels, batch_size = 100, epochs = 10)

Epoch 1/10
114/114 [==============================] - 297s 3s/step - loss: 1.4045 - accuracy: 0.4950
Epoch 2/10
114/114 [==============================] - 294s 3s/step - loss: 0.5879 - accuracy: 0.7940
Epoch 3/10
114/114 [==============================] - 296s 3s/step - loss: 0.3784 - accuracy: 0.8650
Epoch 4/10
114/114 [==============================] - 297s 3s/step - loss: 0.2757 - accuracy: 0.9040
Epoch 5/10
114/114 [==============================] - 294s 3s/step - loss: 0.2203 - accuracy: 0.9237
Epoch 6/10
114/114 [==============================] - 295s 3s/step - loss: 0.1457 - accuracy: 0.9484
Epoch 8/10
114/114 [==============================] - 296s 3s/step - loss: 0.0870 - accuracy: 0.9699
Epoch 9/10
114/114 [==============================] - 297s 3s/step - loss: 0.0985 - accuracy: 0.9635
Epoch 10/10
114/114 [==============================] - 295s 3s/step - loss: 0.0593 - accuracy: 0.9804


**SAVE THE MODEL**

In [35]:
model.save('vegetable_model_ibm.h5')

In [43]:
data_dir = '/home/wsuser/work/Veg-dataset/test_set'

In [44]:
testdata = []

In [45]:
pwd

'/home/wsuser/work'

In [46]:
img_size = 100
def make_data():
    for category in categories:
        path = os.path.join(data_dir, category)
        label = categories.index(category)
        for img_name in os.listdir(path):
            image_path = os.path.join(path, img_name)
            image = cv2.imread(image_path)
            try:
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image,(img_size,img_size))
                image = np.array(image, dtype=np.float32)
                testdata.append([image,label])
            except Exception as e:
                pass  
            
make_data() 

In [47]:
np.random.shuffle(testdata)

In [48]:
tfeatures = []
tlabels = []
for img,label in testdata:
    tfeatures.append(img)
    tlabels.append(label)

In [49]:
tfeatures = np.array(tfeatures, dtype = np.float32)
tlabels = np.array(tlabels)
tfeatures = tfeatures/255.0

In [52]:
model = tf.keras.models.load_model('/home/wsuser/work/vegetable_model_ibm.h5')

**EVALUATE THE MODEL**

In [53]:
model.evaluate(tfeatures, tlabels, verbose = 1)

82/82 [==============================] - 13s 158ms/step - loss: 0.0782 - accuracy: 0.9765


[0.07822700589895248, 0.9765294194221497]

In [54]:
len(testdata)

2599

**PREDICTING**

In [55]:
prediction = model.predict(tfeatures)
count = 0
for i in range(1686):
    if categories[tlabels[i]] == categories[np.argmax(prediction[i]) ]:
        count+=1
print(count,'/',1686)
   

1646 / 1686


In [56]:
for i in range(146):
    print('Actual:' + categories[tlabels[i]] + '\n' + 'Predictions:' + categories[np.argmax(prediction[i])])

Actual:Tomato___Leaf_Mold
Predictions:Tomato___Leaf_Mold
Actual:Pepper,_bell___healthy
Predictions:Pepper,_bell___healthy
Actual:Tomato___Septoria_leaf_spot
Predictions:Tomato___Septoria_leaf_spot
Actual:Tomato___Late_blight
Predictions:Tomato___Late_blight
Actual:Tomato___Bacterial_spot
Predictions:Tomato___Bacterial_spot
Actual:Tomato___Bacterial_spot
Predictions:Tomato___Bacterial_spot
Actual:Tomato___Bacterial_spot
Predictions:Tomato___Bacterial_spot
Actual:Pepper,_bell___healthy
Predictions:Pepper,_bell___healthy
Actual:Potato___Early_blight
Predictions:Potato___Early_blight
Actual:Potato___Late_blight
Predictions:Potato___Late_blight
Actual:Pepper,_bell___healthy
Predictions:Pepper,_bell___healthy
Actual:Pepper,_bell___Bacterial_spot
Predictions:Pepper,_bell___Bacterial_spot
Actual:Tomato___Septoria_leaf_spot
Predictions:Tomato___Septoria_leaf_spot
Actual:Tomato___Leaf_Mold
Predictions:Tomato___Leaf_Mold
Actual:Tomato___Late_blight
Predictions:Tomato___Late_blight
Actual:Potato__

In [57]:
hi = 0
for i in range(1686):
    if categories[tlabels[i]] == categories[np.argmax(prediction[i])]:
        hi = hi + 1
print(hi,'/' + '1686')
print(hi*100/1686,'%')

1646 /1686
97.62752075919336 %


In [58]:
model.save('vegetable_model_ibm.h5')

In [59]:
!tar -zvcf vegetable-leaf-disease-classification-model.tgz vegetable_model_ibm.h5

vegetable_model_ibm.h5


In [60]:
ls -1

Veg-dataset/
vegetable-leaf-disease-classification-model.tgz
vegetable_model_ibm.h5


In [61]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 16.5 MB/s eta 0:00:01


In [62]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
                    "url":"https://us-south.ml.cloud.ibm.com",
                    "apikey" : "E7YrzcF6TgDEbJTXH8TZLdWvcIUDXm6CXdXK9sUHswco"
                  }

In [63]:
client = APIClient(wml_credentials)

In [64]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']['name'] == space_name)['metadata']['id'])

In [65]:
space_uid = guid_from_space_name(client, "Vegetableclassification")
print("Space UID: "+space_uid)

Space UID: 5d381b7f-9c0b-4518-bfe5-bb6b3f4ef8eb


In [66]:
client.set.default_space(space_uid)

'SUCCESS'

In [67]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [68]:
model_details = client.repository.store_model(model = 'vegetable-leaf-disease-classification-model.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"CNN",
    client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid})

In [69]:
model_id = client.repository.get_model_id(model_details)

In [70]:
model_id

'ae8d5055-1228-4fdc-9ff8-09ebe5a94924'

In [71]:
client.repository.download(model_id, 'vegetable-disease-classification-model-new.tar.gz')

Successfully saved model content to file: 'vegetable-disease-classification-model-new.tar.gz'


'/home/wsuser/work/vegetable-disease-classification-model-new.tar.gz'